In [3]:
import os
import glob
import pandas as pd
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator

In [4]:
PATH_CIFAR10 = "logs/CIFAR10"
PATH_MNIST = "logs/MNIST"


In [17]:
def tabulate_events(dpath):
    summary_iterators = [EventAccumulator(os.path.join(dpath, dname)).Reload()  for dname in os.listdir(dpath) if dname.find("events.out.tfevents")==0]

    tags = summary_iterators[0].Tags()['scalars']

    for it in summary_iterators:
        assert it.Tags()['scalars'] == tags

    out = pd.DataFrame()

    for tag in tags:
        steps = []
        values = []

        for events in zip(*[acc.Scalars(tag) for acc in summary_iterators]):
            assert len(set(e.step for e in events)) == 1

            steps.append(events[0].step)
            values.append([e.value for e in events])

        out = pd.concat([out, pd.DataFrame(data=values, index=steps, columns=[tag])], axis=1)

    return out



In [21]:
tabs = tabulate_events(os.path.join("logs/CIFAR10","10"))

In [26]:
for i in range(1,36):
    tabs = tabulate_events(os.path.join(PATH_CIFAR10,str(i)))
    tabs.to_csv(os.path.join(PATH_CIFAR10,str(i)+".csv"))

In [27]:
for i in range(1,36):
    tabs = tabulate_events(os.path.join(PATH_MNIST,str(i)))
    tabs.to_csv(os.path.join(PATH_MNIST,str(i)+".csv"))